#### Import Abstracts

In [24]:
import pandas as pd
data = pd.read_csv("abstractsBig.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,S,label,Abstracts
0,http://yago-knowledge.org/resource/Van_Helsing...,Van Helsing (film),Van Helsing is a 2004 action gothic horror fil...
1,http://yago-knowledge.org/resource/One_Flew_Ov...,One Flew Over the Cuckoo's Nest (film),One Flew Over the Cuckoo's Nest is a 1975 Amer...
2,http://yago-knowledge.org/resource/The_WB,The WB,"The WB Television Network (for Warner Bros., o..."
3,http://yago-knowledge.org/resource/This_Must_B...,This Must Be the Place (film),This Must Be the Place is a 2011 Italian-Frenc...
4,"http://yago-knowledge.org/resource/Monsters,_Inc.","Monsters, Inc.","Monsters, Inc. (also known as Monsters, Incorp..."
...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,"Lincoln College, Oxford","Lincoln College (formally, The College of the ..."
12759,http://yago-knowledge.org/resource/104th_Unite...,104th United States Congress,The 104th United States Congress was a meeting...
12760,http://yago-knowledge.org/resource/London_Busi...,London Business School,London Business School (LBS) is a business sch...
12761,http://yago-knowledge.org/resource/Political_s...,Political science,Political science is the scientific study of p...


#### Model

In [ ]:
pip install git+https://github.com/mmxgn/spacy-clausie.git

In [25]:
import claucy 
import spacy

chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
#nlp_extract = spacy.load("en_core_web_sm")
nlp_sub = spacy.load("en_core_web_sm")
claucy.add_to_pipe(nlp) 
datatrip = []
j = 0
namest = 0
    
for s,label,abstract in data.values:

    if type(abstract) is float:
        datatrip.append([])
        print(j,label)
        j += 1
        continue

    print(j,label)
    doc = nlp(abstract)

    for sent in doc.sents:

        if len(sent._.clauses) == 0:
            continue
      
        propositions = sent._.clauses[0].to_propositions(as_text=False, inflect=None)  
        for triple in propositions:
            if namest == 0:
                name = str(triple[0])
                namest += 1
            if len(triple) == 3:
                sub = nlp_sub(str(triple[0]))
                tokenlist = []
                for token in sub:
                    if token.pos_ == 'NOUN':
                        tokenlist.append(token)
                if any(str(triple[0]).casefold() == str(item) for item in item_list) or any(str(item).casefold() in str(label).casefold() for item in tokenlist) or str(triple[0]) in name or str(triple[0]) in label:
                    trips.append([triple[1], triple[2]])
    
    datatrip.append(trips)
    namest = 0
    trips = []
    j += 1

0 Van Helsing (film)
1 One Flew Over the Cuckoo's Nest (film)
2 The WB
3 This Must Be the Place (film)
4 Monsters, Inc.
5 Paul Thomas Anderson
6 Tim Van Patten
7 Silkeborg IF
8 Bram Stoker
9 Trier
10 Emma Roberts
11 Peter Dinklage
12 Christopher Boyes
13 Joe Jackson (musician)
14 Astralwerks
15 MCA Inc.
16 Ja Rule
17 Grammy Award for Best Male Rock Vocal Performance
18 Liechtenstein national football team
19 Rich Man, Poor Man (miniseries)
20 Monster House (film)
21 Windsor County, Vermont
22 Basingstoke
23 John Stephenson (actor)
24 Arnold Schwarzenegger
25 Carnegie Mellon University
26 Babel (film)
27 City of London
28 RMIT University
29 Satellite Award for Best Original Song
30 Mali
31 Cowpunk
32 Porter County, Indiana
33 Armando Iannucci
34 Orduspor
35 South Somerset
36 Alkmaar
37 Mick Jagger
38 F.C. Hansa Rostock
39 Granada CF
40 London Borough of Camden
41 U.S. Sassuolo Calcio
42 Dubai
43 University of Queensland
44 Michael Shamberg
45 Hugo Award for Best Professional Artist
46 T

In [26]:
datatrip[:5]

[[[is, a 2004 action gothic horror film],
  [stars, Hugh Jackman],
  [is, a homage],
  [is, tribute],
  [Distributed, by Universal Pictures],
  [received, negative reviews]],
 [[is, a 1975 American psychological comedy drama film],
  [features, a supporting cast],
  [Considered, by many to be one of the greatest films ever made],
  [was, the second],
  [won, numerous Golden Globe],
  [won, BAFTA Awards],
  [was deemed, by the United States Library of Congress],
  [was deemed, In 1993],
  [was deemed, culturally, historically, or aesthetically significant"]],
 [[was, an American television network],
  [shut, on September 17, 2006],
  [shut,
   with some programs from both it and competitor UPN (which had shut down on September 15) moving to The CW when it launched the following day, September 18],
  [was closed, in December 2013]],
 [[Be, the Place],
  [stars, Frances McDormand],
  [stars, Sean Penn],
  [deals,
   with a middle-aged wealthy rock star who becomes bored in his retirement 

In [28]:
data['ClCy-Trips-Raw'] = datatrip
data
data.to_csv('ClCyPt1.csv')

In [29]:
data['ClCy-Trips-Raw'][0]

[[is, a 2004 action gothic horror film],
 [stars, Hugh Jackman],
 [is, a homage],
 [is, tribute],
 [Distributed, by Universal Pictures],
 [received, negative reviews]]

In [30]:
from transformers import pipeline
import spacy
import ast

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt', device = -1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0

In [48]:
print(trips)

[['also', 'a 1989 American comedy-drama film'], ['alsoStars', 'John Turturro'], ['alsoStars', 'Richard Edson'], ['stars', 'Ruby Dee']]


In [49]:
trips = []
for s,label,abstracts,triples in data.values[j:]:
    
    if type(abstracts) is float:
        datatrips.append([])
        print(j,label, '- nothing')
        j += 1
        continue

    if j % 1 == 0:          
        print(j,label)
    
    #for triple in ast.literal_eval(triples):
    for triple in triples:
        string = (str(label),str(triple[0]),str(triple[1]))
        string = (' '.join(string))
        tokens = nlp(string)

        for chunk in tokens.noun_chunks:
            chunks.append(chunk.root.head.text)

      # if something in chunk
        if len(chunks) > 0:
            
            if chunks[-1] == 'is' or chunks[-1] == 'was':
                
                ob = nlp(str(triple[1]))
                
              # get noun chunks from object
                for b in ob.noun_chunks:
                    
                    # make triple
                    slm = ("%s. %s %s [MASK] %s" %(string, label, chunks[-1], b.text))
                    # check not already in triples
                    if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                        trips.append([unmasker(slm)[0]['token_str'], b.text])

            else:
                
                ob = nlp(str(triple[1]))
                
                # get noun chunks from object
                for b in ob.noun_chunks:
                    
                    # make triple
                    slm = ("%s. %s [MASK] %s %s" %(string, label, chunks[-1], b.text))
                    
                    # if MASK is punctuation, remove MASK
                    if unmasker(slm)[0]['token_str'] in punc:
                        #print([chunks[-1], b])
                        #print(type(chunks[-1]))
                        #print(type(b.text))
                        if ([chunks[-1], b.text]) not in trips:
                          # check not already in triples
                          trips.append([chunks[-1], b.text])

                    else:
                        if (["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text]) not in trips:
                            # check not already in triples
                            trips.append(["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text])
        else:
            print(j, label, '- different')
            trips.append([str(triple[0]),triple[1]])
          # empty chunk list
        chunks = []

    datatrips.append(trips)
    trips = []
    j += 1

10871 Do the Right Thing
10872 Gordon Willis
10873 Lindy Hemming
10874 Thora Birch
10875 Academy Award for Best Live Action Short Film
10876 Patricia Arquette
10877 Mickey Rooney
10878 Freddy vs. Jason
10879 Harry Potter and the Prisoner of Azkaban (film)
10880 F.C. Copenhagen
10881 War and Peace (1972 TV series)
10882 Troy University
10883 Dream Theater
10884 Elisha Cuthbert
10885 Bristol Rovers F.C.
10886 Edge of Darkness (2010 film)
10887 Jerusalem
10888 Cleopatra Records
10889 Murder by Decree
10890 Arthur Christmas
10891 Allison Janney
10892 Bo Diddley
10893 NBCUniversal
10894 Grateful Dead
10895 The Ghost Writer (film)
10896 Taiwan
10897 Lea Michele
10898 Showgirls
10899 Bhopal
10900 Bright Star (film)
10901 Brown University
10902 Starbucks
10903 Glam punk
10904 Sydney
10905 Michael Piller
10906 Jamie Foxx
10907 Montreal
10908 MGM Resorts International
10909 American Reunion
10910 Skate punk
10911 Washington & Jefferson College
10912 John Steinbeck
10913 Singapore national footba

In [21]:
data['ClCy-Trips-Raw'][1]

[['is', /jeɪ/ YAY],
 [regarded, as one of the most influential hip hop artists and producers],
 [regarded, widely],
 [Born, in Atlanta],
 [released, his debut studio album],
 [founded, the record label],
 [explored, diverse musical elements],
 [diversified, his musical styles],
 [diversified, Ye],
 [released, its sequel],
 [holds, the joint record],
 [has collaborated,
  with Nike, Louis Vuitton, Gap, and A.P.C. on clothing and footwear],
 [has collaborated, As a fashion designer],
 [is, the founder],
 [is, head],
 [launched, an unsuccessful independent presidential campaign],
 [condemned, widely],
 [condemned, In 2022],
 [condemned, after making a series of antisemitic statements],
 [announced, his 2024 presidential campaign],
 [praised, Adolf Hitler]]

In [54]:
data['ClCy'] = datatrips
data

,S,label,Abstracts,ClCy-Trips-Raw,ClCy
0,http://yago-knowledge.org/resource/Van_Helsing...,Van Helsing (film),Van Helsing is a 2004 action gothic horror fil...,"[[(is), (a, 2004, action, gothic, horror, film...","[[also, a 2004 action gothic horror film], [st..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,One Flew Over the Cuckoo's Nest (film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[[(is), (a, 1975, American, psychological, com...","[[also, a 1975 American psychological comedy d..."
2,http://yago-knowledge.org/resource/The_WB,The WB,"The WB Television Network (for Warner Bros., o...","[[(was), (an, American, television, network)],...","[[not, an American television network], [close..."
3,http://yago-knowledge.org/resource/This_Must_B...,This Must Be the Place (film),This Must Be the Place is a 2011 Italian-Frenc...,"[[(Be), (the, Place)], [(stars), (Frances, McD...","[[toBe, the Place], [stars, Frances McDormand]..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.","Monsters, Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[[is, (Inc.)], [(believes), (are)], [(believes...","[[not, Inc.], [premierAt, the El Capitan Theat..."
...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,"Lincoln College, Oxford","Lincoln College (formally, The College of the ...","[[is, (The, College)], [(was, founded), (in, 1...","[[also, The College], [birthplaceOf, Richard F..."
12759,http://yago-knowledge.org/resource/104th_Unite...,104th United States Congress,The 104th United States Congress was a meeting...,"[[(was), (a, meeting)], [(met), (from, January...","[[not, a meeting], [metFrom, January], [metWas..."
12760,http://yago-knowledge.org/resource/London_Busi...,London Business School,London Business School (LBS) is a business sch...,"[[is, (LBS)], [(acquired), (the, Marylebone, T...","[[lbs, LBS], [alsoAcquired, the Marylebone Tow..."
12761,http://yago-knowledge.org/resource/Political_s...,Political science,Political science is the scientific study of p...,"[[(is), (the, scientific, study)], [(is), (a, ...","[[also, the scientific study], [also, a social..."


# final processing 

In [72]:
final_trips = []
import nltk
import ast
trips = []
k = 0
for s,label,abstract,_,triples in data.values:
    print(k)
    k+=1
    if type(abstract) == float:
        print('ye')
        final_trips.append([])
        continue
    
    #for triple in ast.literal_eval(triples):
    for triple in triples:
        if triple[0] == 'is' or triple[0] == 'also' or triple[0] == 'an' or triple[0] == 'a' or triple[0] == 'was' or triple[0] == 'as' or triple[0] == 'not':
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(triple[0],'- type'," ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append(['type'," ".join(simpletok[1:])])
            else:
                print(triple[0], '-', 'type', triple[1], '-', label)
                trips.append(['type', triple[1]])
        else:
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(" ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append([triple[0], " ".join(simpletok[1:])])
            else:
                trips.append(triple)
    final_trips.append(trips)
    trips = []

0
1
2
not - type American_television_network - The WB
3
4
not - type Inc. - Monsters, Inc.
5
6
7
8
9
not - type city - Trier
10
11
12
13
14
15
16
17
18
19
20
21
not - type county - Windsor County, Vermont
22
not - type zing-stohk - Basingstoke
not - type part - Basingstoke
23
24
25
26
27
not - type city - City of London
28
29
30
not - type landlocked_country - Mali
31
not - type subgenre - Cowpunk
32
not - type county - Porter County, Indiana
not - type part - Porter County, Indiana
33
34
35
36
37
38
ye
39
40
41
42
43
44
45
46
47
48
not - type professional_football_club - Ross County F.C.
49
50
51
52
53
54
55
56
57
58
59
60
not - type mə-NOT-ə - Lady Gaga
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
not - type county - Prince George's County, Maryland
90
91
92
not - type 2006_American_documentary_film - An Inconvenient Truth
93
not - type member - Paquito D'Rivera
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115

In [73]:
len(final_trips)

12763

In [74]:
import re
trips = []
fin = []
for triples in final_trips:
    for triple in triples:
        pred = re.sub( r"([A-Z])", r" \1", triple[0]).split()
        if len(pred) > 1:
            if pred[0] == 'also' or pred[0] == 'was' or pred[0] == 'also' or pred[0] == 'has':
                relation = pred[1].lower()
                simpletok = str(triple[1]).split(' ')
                if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also' or simpletok[0] == 'his' or simpletok[0] == 'she':
                    if len(" ".join(simpletok[1:])) == 0:
                        continue
                        #print(triple[0], triple[1], '- Removed' )
                        #print()
                    else:
                        #print('fix')
                        #print(triple)
                        #print(relation," ".join(simpletok[1:]))
                        #print()
                        trips.append([relation," ".join(simpletok[1:])])
                else:
                    trips.append([relation,triple[1]])
            else:
                trips.append(triple)
        else:
            trips.append(triple)
    fin.append(trips)
    trips = []
fin

[[['type', '2004_action_gothic_horror_film'],
  ['stars', 'Hugh_Jackman'],
  ['type', 'homage'],
  ['distributedBy', 'Universal_Pictures'],
  ['received', 'negative_reviews']],
 [['type', '1975_American_psychological_comedy_drama_film'],
  ['features', 'supporting_cast'],
  ['oneOf', 'greatest_films'],
  ['won', 'numerous_Golden_Globe'],
  ['won', 'BAFTA_Awards'],
  ['divisionOf', 'United_States_Library'],
  ['libraryOf', 'Congress']],
 [['type', 'American_television_network'],
  ['closedOn', 'September'],
  ['launched', 'some_programs'],
  ['launched', 'both_it'],
  ['launched', 'competitor_UPN'],
  ['launched', 'which'],
  ['officiallyLaunched', 'September'],
  ['launched', 'The_CW'],
  ['launched', 'following_day'],
  ['closedIn', 'December']],
 [['toBe', 'Place'],
  ['stars', 'Frances_McDormand'],
  ['stars', 'Sean_Penn'],
  ['dealsIn', 'middle-aged_wealthy_rock_star'],
  ['believesIn', 'who'],
  ['boredIn', 'his_retirement'],
  ['continuesIn', 'quest'],
  ['findsIn', "his_father's

In [75]:
trips = []
done = []
nouns = []
import spacy 
nlp = spacy.load("en_core_web_sm")

for triples in fin: 
    for triple in triples:
        dd = triple[0].split(' ')
        if len(dd) > 1:
      #print(triple[0], '-', label, triple[0].split(' ')[1], triple[1])
            tokens = nlp(str(triple[1]))
            for token in tokens.noun_chunks:
                nouns.append(token)
            if len(nouns) == 1:
                print(triple[0], '-', label, triple[0].split(' ')[1], nouns[0])
                trips.append([triple[0].split(' ')[1], nouns[0]])
            else:
                tokenized = nltk.word_tokenize(str(triple[1]))
                pos = nltk.pos_tag(tokenized)
                for i in pos:
                    if i[1] == 'CD' or i[1] == 'NN':
                        print(triple[0], '-', label, triple[0].split(' ')[1], i[0])
                        trips.append([triple[0].split(' ')[1], i[0]])
            nouns = []
        else:
            trips.append(triple)
    done.append(trips)
    trips = []

In [76]:
trips = []
final = []
for triples in done:
    for triple in triples:
        obj = str(triple[1]).split(" ")
        obj = "_".join(obj)
        trips.append([triple[0], obj])
    final.append(trips)
    trips = []
len(final), final[:5]

(12763,
 [[['type', '2004_action_gothic_horror_film'],
   ['stars', 'Hugh_Jackman'],
   ['type', 'homage'],
   ['distributedBy', 'Universal_Pictures'],
   ['received', 'negative_reviews']],
  [['type', '1975_American_psychological_comedy_drama_film'],
   ['features', 'supporting_cast'],
   ['oneOf', 'greatest_films'],
   ['won', 'numerous_Golden_Globe'],
   ['won', 'BAFTA_Awards'],
   ['divisionOf', 'United_States_Library'],
   ['libraryOf', 'Congress']],
  [['type', 'American_television_network'],
   ['closedOn', 'September'],
   ['launched', 'some_programs'],
   ['launched', 'both_it'],
   ['launched', 'competitor_UPN'],
   ['launched', 'which'],
   ['officiallyLaunched', 'September'],
   ['launched', 'The_CW'],
   ['launched', 'following_day'],
   ['closedIn', 'December']],
  [['toBe', 'Place'],
   ['stars', 'Frances_McDormand'],
   ['stars', 'Sean_Penn'],
   ['dealsIn', 'middle-aged_wealthy_rock_star'],
   ['believesIn', 'who'],
   ['boredIn', 'his_retirement'],
   ['continuesIn', 

In [77]:
data['ClCy'] = final
data

,S,label,Abstracts,ClCy-Trips-Raw,ClCy
0,http://yago-knowledge.org/resource/Van_Helsing...,Van Helsing (film),Van Helsing is a 2004 action gothic horror fil...,"[[(is), (a, 2004, action, gothic, horror, film...","[[type, 2004_action_gothic_horror_film], [star..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,One Flew Over the Cuckoo's Nest (film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[[(is), (a, 1975, American, psychological, com...","[[type, 1975_American_psychological_comedy_dra..."
2,http://yago-knowledge.org/resource/The_WB,The WB,"The WB Television Network (for Warner Bros., o...","[[(was), (an, American, television, network)],...","[[type, American_television_network], [closedO..."
3,http://yago-knowledge.org/resource/This_Must_B...,This Must Be the Place (film),This Must Be the Place is a 2011 Italian-Frenc...,"[[(Be), (the, Place)], [(stars), (Frances, McD...","[[toBe, Place], [stars, Frances_McDormand], [s..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.","Monsters, Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[[is, (Inc.)], [(believes), (are)], [(believes...","[[type, Inc.], [premierAt, El_Capitan_Theatre]..."
...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,"Lincoln College, Oxford","Lincoln College (formally, The College of the ...","[[is, (The, College)], [(was, founded), (in, 1...","[[type, The_College], [birthplaceOf, Richard_F..."
12759,http://yago-knowledge.org/resource/104th_Unite...,104th United States Congress,The 104th United States Congress was a meeting...,"[[(was), (a, meeting)], [(met), (from, January...","[[type, meeting], [metFrom, January], [metWash..."
12760,http://yago-knowledge.org/resource/London_Busi...,London Business School,London Business School (LBS) is a business sch...,"[[is, (LBS)], [(acquired), (the, Marylebone, T...","[[lbs, LBS], [acquired, Marylebone_Town_Hall]]"
12761,http://yago-knowledge.org/resource/Political_s...,Political science,Political science is the scientific study of p...,"[[(is), (the, scientific, study)], [(is), (a, ...","[[type, scientific_study], [type, social_scien..."


In [78]:
data.to_csv('ClCyBig.csv')

In [ ]:
pip install rfc3987

In [79]:
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF
from rdflib import Literal, XSD
from rfc3987 import parse
flag = True
g = Graph()
k = 0
n = 0
for s,label,abstract,_,triples in data.values:
    
    if type(abstract) == float:
        continue
        
    for triple in triples:
        flag = True
        
        try:
            int(triple[1])
        except:
            flag = False 
    
        if flag is True:
            rel = triple[0].replace("'", '')
            rel = rel.replace('"', '')
            rel = rel.replace('`', '')
            rel = rel.replace('.', '')
            schema = ('http://schema.org/%s' %(rel))
            year = int(triple[1])
            year = Literal(year, datatype=XSD.gYear)
            try:
                parse(schema, rule='IRI')
            except:
                n+=1
                print(n,'bad -',triple)
                continue
            g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), year ))
            k+=1
        else:
            obj = triple[1].replace("'", '')
            obj = obj.replace('"', '')
            obj = obj.replace('`', '')
            obj = obj.replace('.', '')
            link = 'http://yago-knowledge.org/resource/%s' %(obj)
            if triple[0] == 'type':
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(link)
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), RDF.type, rdflib.URIRef(link)) )
                k+=1
            else:
                rel = triple[0].replace("'", '')
                rel = rel.replace('"', '')
                rel = rel.replace('`', '')
                rel = rel.replace('.', '')
                schema = ('http://schema.org/%s' %(rel))
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                try:
                    parse(schema, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), rdflib.URIRef(link)) )
                k+=1

1 bad - ['[UNK]', '深作']
2 bad - ['##sKielce', 'Krosno_Voivodeships']
3 bad - ['##sMeet', 'MTV_reality_series']
4 bad - ['isPronounced', 'pronounced_[əmɪˈt̪ɑːbʱ_ˈbətːʃən']
5 bad - ['[UNK]', '福島県']
6 bad - ['##nsWere', 'sovereign_nation']
7 bad - ['accountsOf', 'only_1.5%']
8 bad - ['##sOf', 'some_legal_systems']
9 bad - ['##sOf', 'those']
10 bad - ['##sOf', 'Scotland']
11 bad - ['##sOf', 'South_Africa']
12 bad - ['##sOf', 'Scandinavia']
13 bad - ['##sOf', 'Pakistan']
14 bad - ['##sOf', 'India']
15 bad - ['##sOf', 'Bangladesh']
16 bad - ['##sOf', 'British_Crown_dependencies']
17 bad - ['##sOf', 'Jersey']
18 bad - ['##sOf', 'Guernsey']
19 bad - ['##sOf', 'Isle']
20 bad - ['##sOf', 'Man']
21 bad - ['[UNK]', '埼玉県']
22 bad - ['[UNK]', '千葉県']
23 bad - ['[UNK]', '静岡県']
24 bad - ['[UNK]', '能登']
25 bad - ['##sSpelled', 'super-model_or_super_model']
26 bad - ['##sStarted', 'their_career']
27 bad - ['[UNK]', '日本語']
28 bad - ['##nKing', 'rule']
29 bad - ['##nKing', 'first_Tongan_king']
30 bad - ['#

In [80]:
total = 0
for triples in data['ClCy']:
    if len(triples) == 0:
        continue 
    else:
        for triple in triples:
            total += 1
total,k

(111281, 110967)

In [82]:
n,total, n/total

(314, 111281, 0.0028216856426523844)

314/111374 triples were not in proper format and needed to be deleted, thats 0.3%

In [84]:
g.serialize(destination='clcyBig.ttl', format='turtle')

<Graph identifier=N20fccd2d17b848fbb019acfdc209c21b (<class 'rdflib.graph.Graph'>)>

In [83]:
max_trips = 0
for s,l,a,r,t in data.values:
    max_trips = max(len(t), max_trips)
    if max_trips == len(t):
        print(l,'-',len(t))

Van Helsing (film) - 5
One Flew Over the Cuckoo's Nest (film) - 7
The WB - 10
This Must Be the Place (film) - 14
Paul Thomas Anderson - 15
Trier - 16
Ja Rule - 18
Arnold Schwarzenegger - 18
Mick Jagger - 27
Angel (1999 TV series) - 28
Shaquille O'Neal - 38
Cary Grant - 38
Frank Sinatra - 39
Beauty and the Beast (1991 film) - 48
Kentucky - 48
John Kerry - 53
